In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spam Checking").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/02/01 22:03:16 WARN Utils: Your hostname, CuongAtPoland resolves to a loopback address: 127.0.1.1; using 172.27.34.142 instead (on interface eth0)
24/02/01 22:03:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/01 22:03:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_location = "./spamham.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("quote", '"') \
  .option("escape", '"') \
  .load(file_location)


In [4]:
df.show()

+--------------------+----+
|                text|spam|
+--------------------+----+
|Subject: naturall...|   1|
|Subject: the stoc...|   1|
|Subject: unbeliev...|   1|
|Subject: 4 color ...|   1|
|Subject: do not h...|   1|
|Subject: great nn...|   1|
|Subject: here ' s...|   1|
|Subject: save you...|   1|
|Subject: undelive...|   1|
|Subject: save you...|   1|
|Subject: las vega...|   1|
|Subject: save you...|   1|
|Subject: brighten...|   1|
|Subject: wall str...|   1|
|Subject: fpa noti...|   1|
|Subject: search e...|   1|
|Subject: only our...|   1|
|Subject: localize...|   1|
|Subject: security...|   1|
|Subject: 21 st ce...|   1|
+--------------------+----+
only showing top 20 rows



In [5]:
df.count()

5728

In [6]:
def elim_subject(x):
  return ''.join(x.split(':')[1:]).strip()

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

elim_subject_udf = udf(elim_subject, StringType())
df_elim = df.withColumn('message', elim_subject_udf(df['text']))
df_elim.show(100)

+--------------------+----+--------------------+
|                text|spam|             message|
+--------------------+----+--------------------+
|Subject: naturall...|   1|naturally irresis...|
|Subject: the stoc...|   1|the stock trading...|
|Subject: unbeliev...|   1|unbelievable new ...|
|Subject: 4 color ...|   1|4 color printing ...|
|Subject: do not h...|   1|do not have money...|
|Subject: great nn...|   1|great nnews  hell...|
|Subject: here ' s...|   1|here ' s a hot pl...|
|Subject: save you...|   1|save your money b...|
|Subject: undelive...|   1|undeliverable  ho...|
|Subject: save you...|   1|save your money b...|
|Subject: las vega...|   1|las vegas high ri...|
|Subject: save you...|   1|save your money b...|
|Subject: brighten...|   1|brighten those te...|
|Subject: wall str...|   1|wall street pheno...|
|Subject: fpa noti...|   1|fpa notice  ebay ...|
|Subject: search e...|   1|search engine pos...|
|Subject: only our...|   1|only our software...|
|Subject: localize..

In [7]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol='message', outputCol='words')
wordsData = tokenizer.transform(df_elim)
wordsData.show()

+--------------------+----+--------------------+--------------------+
|                text|spam|             message|               words|
+--------------------+----+--------------------+--------------------+
|Subject: naturall...|   1|naturally irresis...|[naturally, irres...|
|Subject: the stoc...|   1|the stock trading...|[the, stock, trad...|
|Subject: unbeliev...|   1|unbelievable new ...|[unbelievable, ne...|
|Subject: 4 color ...|   1|4 color printing ...|[4, color, printi...|
|Subject: do not h...|   1|do not have money...|[do, not, have, m...|
|Subject: great nn...|   1|great nnews  hell...|[great, nnews, , ...|
|Subject: here ' s...|   1|here ' s a hot pl...|[here, ', s, a, h...|
|Subject: save you...|   1|save your money b...|[save, your, mone...|
|Subject: undelive...|   1|undeliverable  ho...|[undeliverable, ,...|
|Subject: save you...|   1|save your money b...|[save, your, mone...|
|Subject: las vega...|   1|las vegas high ri...|[las, vegas, high...|
|Subject: save you..

In [8]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol='words', outputCol='features')
model = cv.fit(wordsData)
featurizedDF = model.transform(wordsData)

featurizedDF.show()

+--------------------+----+--------------------+--------------------+--------------------+
|                text|spam|             message|               words|            features|
+--------------------+----+--------------------+--------------------+--------------------+
|Subject: naturall...|   1|naturally irresis...|[naturally, irres...|(37440,[0,1,2,3,4...|
|Subject: the stoc...|   1|the stock trading...|[the, stock, trad...|(37440,[0,4,7,19,...|
|Subject: unbeliev...|   1|unbelievable new ...|[unbelievable, ne...|(37440,[0,1,2,3,4...|
|Subject: 4 color ...|   1|4 color printing ...|[4, color, printi...|(37440,[0,1,2,3,6...|
|Subject: do not h...|   1|do not have money...|[do, not, have, m...|(37440,[0,2,3,4,5...|
|Subject: great nn...|   1|great nnews  hell...|[great, nnews, , ...|(37440,[0,1,2,3,4...|
|Subject: here ' s...|   1|here ' s a hot pl...|[here, ', s, a, h...|(37440,[0,1,2,3,4...|
|Subject: save you...|   1|save your money b...|[save, your, mone...|(37440,[0,1,2,4,5...|

In [9]:
from pyspark.ml.feature import IDF
from pyspark.sql.functions import col

idf = IDF(inputCol="features", outputCol="idfFeatures")
idfModel = idf.fit(featurizedDF)
rescaledData = idfModel.transform(featurizedDF)
#display(rescaledData)

dataForTraining = rescaledData.select(col('spam').alias('label'), col('idfFeatures').alias('features'))
dataForTraining.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|(37440,[0,1,2,3,4...|
|    1|(37440,[0,4,7,19,...|
|    1|(37440,[0,1,2,3,4...|
|    1|(37440,[0,1,2,3,6...|
|    1|(37440,[0,2,3,4,5...|
|    1|(37440,[0,1,2,3,4...|
|    1|(37440,[0,1,2,3,4...|
|    1|(37440,[0,1,2,4,5...|
|    1|(37440,[0,1,2,3,4...|
|    1|(37440,[0,1,2,4,5...|
|    1|(37440,[0,1,2,3,4...|
|    1|(37440,[0,1,2,4,5...|
|    1|(37440,[0,2,3,4,5...|
|    1|(37440,[0,1,2,3,4...|
|    1|(37440,[0,1,2,3,4...|
|    1|(37440,[0,2,3,4,5...|
|    1|(37440,[0,1,2,3,5...|
|    1|(37440,[0,1,2,3,5...|
|    1|(37440,[0,1,9,27,...|
|    1|(37440,[0,1,2,3,4...|
+-----+--------------------+
only showing top 20 rows



24/02/01 22:06:59 WARN DAGScheduler: Broadcasting large task binary with size 1003.2 KiB


In [10]:
trainDF, testDF = dataForTraining.randomSplit([0.8, 0.2])
print(trainDF.count())
print(testDF.count())

24/02/01 22:07:15 WARN DAGScheduler: Broadcasting large task binary with size 1016.4 KiB
24/02/01 22:07:16 WARN DAGScheduler: Broadcasting large task binary with size 1016.4 KiB


4580


1148


In [11]:
import numpy as np
regParamValues = np.linspace(0.3, 0.01, 6)
print(regParamValues)

[0.3   0.242 0.184 0.126 0.068 0.01 ]


In [12]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np

lr = LogisticRegression(maxIter = 3)

regParamValues = np.linspace(0.3, 0.01, 2)
print(regParamValues)
elasticNetParamValues = np.linspace(0.3, 0.8, 2)
print(elasticNetParamValues)

paramGr_LR = ParamGridBuilder().addGrid(lr.regParam, regParamValues)\
  .addGrid(lr.elasticNetParam, elasticNetParamValues)\
  .build()

crossValid_LR = CrossValidator(estimator=lr, estimatorParamMaps=paramGr_LR, evaluator=BinaryClassificationEvaluator(), numFolds=5)
cvModel = crossValid_LR.fit(trainDF)
best_model_lr = cvModel.bestModel.summary
print(best_model_lr.predictions.columns)

[0.3  0.01]
[0.3 0.8]


24/02/01 22:07:39 WARN DAGScheduler: Broadcasting large task binary with size 1022.9 KiB
24/02/01 22:07:40 WARN DAGScheduler: Broadcasting large task binary with size 1061.4 KiB
24/02/01 22:07:41 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/02/01 22:07:41 WARN DAGScheduler: Broadcasting large task binary with size 1062.1 KiB
24/02/01 22:07:41 WARN DAGScheduler: Broadcasting large task binary with size 1062.1 KiB
24/02/01 22:07:41 WARN DAGScheduler: Broadcasting large task binary with size 1062.1 KiB
24/02/01 22:07:41 WARN DAGScheduler: Broadcasting large task binary with size 1062.1 KiB
24/02/01 22:07:41 WARN DAGScheduler: Broadcasting large task binary with size 1062.1 KiB
24/02/01 22:07:42 WARN DAGScheduler: Broadcasting large task binary with size 1062.1 KiB
24/02/01 22:07:42 WARN DAGScheduler: Broadcasting large task binary with size 1062.1 KiB
24/02/01 22:07:42 WARN DAGScheduler: Broadcasting large task binary with size 1022.8 KiB
24/

['label', 'features', 'rawPrediction', 'probability', 'prediction']


In [13]:
my_eval_lr = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
areaUnderROC = my_eval_lr.evaluate(best_model_lr.predictions)
print(areaUnderROC)

24/02/01 22:08:13 WARN DAGScheduler: Broadcasting large task binary with size 1129.1 KiB


0.8831818181818182


In [14]:
train_fit_lr = best_model_lr.predictions.select('label','prediction')
train_fit_lr.groupBy('label', 'prediction').count().show()

24/02/01 22:08:24 WARN DAGScheduler: Broadcasting large task binary with size 1139.4 KiB


+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  843|
|  1.0|       0.0|  257|
|  0.0|       0.0| 3480|
+-----+----------+-----+



24/02/01 22:08:25 WARN DAGScheduler: Broadcasting large task binary with size 1124.3 KiB
